In [1]:
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import OneHotEncoder
!pip install --upgrade category_encoders
import category_encoders  as ce
from matplotlib import pyplot as plt

drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.3 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
data_frame = pd.read_csv("/content/drive/MyDrive/Organizacion De Datos/trabajo_practico_2/secondary_data.csv",sep=";")

In [3]:
data_frame.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w


Para aquellas filas con mucho porcentaje de nans ***(a mi criterio, las que tienen+60%)*** , voy a fillearlas con 'unknown', mientras que aquellas que tengan porcentaje bajo, voy a usar el **'most-frequent strategy'** de simpleImputer

In [4]:
(( len(data_frame) - data_frame.count() )*100 / len(data_frame)).nlargest(len(data_frame.columns))

veil-type               94.797688
spore-print-color       89.595376
veil-color              87.861272
stem-root               84.393064
stem-surface            62.427746
gill-spacing            41.040462
cap-surface             23.121387
gill-attachment         16.184971
ring-type                4.046243
class                    0.000000
stem-color               0.000000
habitat                  0.000000
has-ring                 0.000000
stem-width               0.000000
cap-diameter             0.000000
stem-height              0.000000
gill-color               0.000000
does-bruise-or-bleed     0.000000
cap-color                0.000000
cap-shape                0.000000
season                   0.000000
dtype: float64

# Separando la variable a predecir

In [5]:
X = data_frame.drop('class', axis=1)
y = data_frame['class']

In [6]:
y = y.map({"e":1,"p":0})

# Division de sets 

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
X_validation, X_test, y_validation, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=13)

guardo el indice ya que despues de usar el SimpleImputer se me pierde ( lo necesito para mean encoding de category_encoders ) 

In [8]:
X_train_saved_index =  X_train.index
X_validation_saved_index = X_validation.index
X_test_saved_index = X_test.index

# Imputacion

`Imputacion 1) Reemplazar por una constante`

In [9]:
X_train[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ] = X_train[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ].fillna('unknown')
X_validation[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ] = X_validation[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ].fillna('unknown')
X_test[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ] = X_test[ ['veil-type','spore-print-color','veil-color','stem-root','stem-surface'] ].fillna('unknown')

`Imputacion 2) Reemplazar por el mas frecuente`

In [10]:
from sklearn.impute import SimpleImputer
simple = SimpleImputer(strategy='most_frequent')

In [11]:
X_train = pd.DataFrame(simple.fit_transform(X_train), columns = X_train.columns)
X_validation = pd.DataFrame(simple.transform(X_validation), columns = X_validation.columns)
X_test = pd.DataFrame(simple.transform(X_test), columns = X_test.columns)

In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48855 entries, 0 to 48854
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   cap-diameter          48855 non-null  object
 1   cap-shape             48855 non-null  object
 2   cap-surface           48855 non-null  object
 3   cap-color             48855 non-null  object
 4   does-bruise-or-bleed  48855 non-null  object
 5   gill-attachment       48855 non-null  object
 6   gill-spacing          48855 non-null  object
 7   gill-color            48855 non-null  object
 8   stem-height           48855 non-null  object
 9   stem-width            48855 non-null  object
 10  stem-root             48855 non-null  object
 11  stem-surface          48855 non-null  object
 12  stem-color            48855 non-null  object
 13  veil-type             48855 non-null  object
 14  veil-color            48855 non-null  object
 15  has-ring              48855 non-null

acá vuelvo a pasar las medidas a float porque se me pasaron a objeto

In [13]:
X_train[ ['cap-diameter', 'stem-height', 'stem-width'] ] = X_train[ ['cap-diameter', 'stem-height', 'stem-width'] ].astype(float)
X_validation[ ['cap-diameter', 'stem-height', 'stem-width'] ] = X_validation[ ['cap-diameter', 'stem-height', 'stem-width'] ].astype(float)
X_test[ ['cap-diameter', 'stem-height', 'stem-width'] ] = X_test[ ['cap-diameter', 'stem-height', 'stem-width'] ].astype(float)

reemplazo el indice por el original

In [14]:
X_train.index = X_train_saved_index
X_validation.index = X_validation_saved_index 
X_test.index = X_test_saved_index

# Encodings

Mean Encoding

In [15]:
meanEncod = ce.TargetEncoder(cols = ['cap-surface'	, 'gill-attachment'	])

In [16]:
X_train = meanEncod.fit_transform(X_train, y_train)
X_validation = meanEncod.transform(X_validation)
X_test =  meanEncod.transform(X_test)

OHE Encoding

In [17]:
ohe_ring = OneHotEncoder()

encoded_ring_train = ohe_ring.fit_transform(X_train[['has-ring']]).todense().astype(int)
encoded_ring_validation = ohe_ring.transform(X_validation[['has-ring']]).todense().astype(int)
encoded_ring_test = ohe_ring.transform(X_test[['has-ring']]).todense().astype(int)

encoded_bruise_train = ohe_ring.fit_transform(X_train[['does-bruise-or-bleed']]).todense().astype(int)
encoded_bruise_validation = ohe_ring.transform(X_validation[['does-bruise-or-bleed']]).todense().astype(int)
encoded_bruise_test = ohe_ring.transform(X_test[['does-bruise-or-bleed']]).todense().astype(int)

In [18]:
X_train['has-ring'] = encoded_ring_train
X_validation['has-ring'] = encoded_ring_validation
X_test['has-ring'] = encoded_ring_test

X_train['does-bruise-or-bleed'] = encoded_bruise_train
X_validation['does-bruise-or-bleed'] = encoded_bruise_validation
X_test['does-bruise-or-bleed'] = encoded_bruise_test

binaryEncoding

In [19]:
bin_encoded = ce.BinaryEncoder(cols=["cap-shape","cap-color","gill-spacing","gill-color","stem-root","stem-surface",
                                     "stem-color","veil-type","veil-color","ring-type","spore-print-color","habitat" ,"season"], return_df = True)

X_train = bin_encoded.fit_transform(X_train)
X_validation = bin_encoded.transform(X_validation)
X_test = bin_encoded.transform(X_test)

# XGBoost

In [20]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn import tree

## Busqueda de hiper-parametros

In [21]:
from sklearn.model_selection import RandomizedSearchCV

In [22]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic')
param_dist = {
      'n_estimators':np.arange(10, 120, 10),
      'learning_rate' : [0.05,0.10,0.15,0.20,0.25,0.30],
      'max_depth' : [ 3, 4, 5, 6, 8, 10, 12, 15],
      'min_child_weight' : [ 1, 3, 5, 7 ],
      'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
      'colsample_bytree' : [ 0.3, 0.4, 0.5 , 0.7 ]
}

In [23]:
rs_xg_class = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter=5, random_state = 13, scoring='roc_auc')

## Entrenamos y predecimos

In [24]:
rs_xg_class.fit(X_train, y_train)
preds = rs_xg_class.predict(X_validation)

## Mejores hiper-parametros

In [25]:
rs_xg_class.best_params_

{'n_estimators': 80,
 'min_child_weight': 3,
 'max_depth': 8,
 'learning_rate': 0.3,
 'gamma': 0.0,
 'colsample_bytree': 0.7}

## Mejor modelo

In [26]:
model = rs_xg_class.best_estimator_

In [27]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=80, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

Matriz de confusión

In [28]:
cm = confusion_matrix(y_validation, preds)
cm

array([[3374,    0],
       [   0, 2733]])

## **Score para validation**

In [29]:
auc_result_XGBoost = roc_auc_score(y_validation, model.predict_proba(X_validation)[:,1])
print("AUC_ROC: %f" % (auc_result_XGBoost))

AUC_ROC: 1.000000


## **Score final en Test**

In [30]:
auc_result_XGBoost_test_only = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
print("AUC_ROC: %f" % (auc_result_XGBoost_test_only))

AUC_ROC: 1.000000


# Guardado del modelo con pickle

In [31]:
import pickle
pickle.dump(model, open('/content/drive/MyDrive/Organizacion De Datos/trabajo_practico_2/modelos/XGBoost.pkl', 'wb'))

# PREGUNTAS 

## ¿Cuál es el score en test?

In [32]:
print("AUC_ROC: %f" % (auc_result_XGBoost_test_only))

AUC_ROC: 1.000000


### CSV con predicciones

In [33]:
compare_df = pd.DataFrame(y_test)

In [34]:
y_predict_test = model.predict(X_test)

In [35]:
compare_df['predicted'] = y_predict_test

In [36]:
compare_df.columns = ['actual_class' , 'predicted_class']
compare_df = compare_df.drop(['actual_class'], axis=1)

In [37]:
compare_df

,predicted_class
37726,0
10734,0
44627,0
39270,0
22460,1
...,...
38769,0
2800,1
42584,1
23210,0


In [38]:
from pathlib import Path  
filepath = Path('/content/drive/MyDrive/Organizacion De Datos/trabajo_practico_2/predicciones_XGBoost.csv')  
compare_df.to_csv(filepath,index=True)

## ¿Por qué cree que logro/no logro el mismo valor de AUC que con Random Forest?

Para el set de validation, llegué a un AUC_ROC = 1 en ambos casos. Pero para el de set, el random forest dió ligeramete menor (0.9998). Esto probablemente tenga que ver con que **XGBoost** (según entendí) funciona de una manera tal que va corrigiendo los errores del arbol anterior (boosting). Mientras que **RandomForest**, el modelo aprende diferentes correlaciones entre disntintas features de manera random y promedia los resultados.(bagging) Lo cual puede significar que el XGBoost proporciona "más precisión".